In [ ]:
import rasterio
from vis_tools import read_hysplit_netcdf
import numpy as np
import os
import colorcet as cc

FILENAME = '18042918_taupo_15.0_0.01.nc'

temp = read_hysplit_netcdf(FILENAME).isel(time=slice(1, None)).transpose('time', 'lat', 'lon')  # remove trivial t=0 step
np.place(temp['total_deposition'].values, np.isnan(temp['total_deposition'].values), 0)  # transform all NaN values to 0
temp.to_netcdf('temp.nc')

model = rasterio.open('temp.nc', 'r')

cmap = []
for i in range(256):
    cmap.append(cc.m_fire_r(1))

for band in model.indexes:

    new_dataset = rasterio.open(FILENAME.rstrip('.nc')+'_{}{}'.format(band, '.tif'), 'w',
                                driver='GTiff',
                                height=model.height, width=model.width,
                                count=1, dtype=model.profile['dtype'],
                                crs={'init': 'epsg:4326'}, transform=model.affine,
                                nodata=0)
    new_dataset.write(model.read(band), 1)
    new_dataset.close()
    print('band {} is written'.format(band))

model.close()
os.remove('temp.nc')
print('done')

In [ ]:
import simplekml
import rasterio

FILENAME = '18042918_taupo_15.0_0.01_1.tif'

ts = int(FILENAME.split('.')[-2].split('_')[-1])

kml = simplekml.Kml()

image = kml.newgroundoverlay(name='time_step_{}'.format(ts))
image.icon.href = kml.addfile(FILENAME)
with rasterio.open(FILENAME, 'r') as src:
    image.latlonbox.north = src.bounds.top
    image.latlonbox.south = src.bounds.bottom
    image.latlonbox.east = src.bounds.right
    image.latlonbox.west = src.bounds.left
    
kml.savekmz('ash.kmz')